In [38]:
from brtdevkit.data import Dataset
import numpy as np
import pandas as pd

In [54]:
geohash_df = pd.read_csv(filepath_or_buffer='/data/jupiter/alex.li/20240212_geohash_table_v7_0_rc.csv', index_col="Unnamed: 0")
geohash_train_df = geohash_df[geohash_df['bucket'] == 'train']
geohash_test_df = geohash_df[geohash_df['bucket'] == 'test']
new_geohashes = set()
def filter_df(df_orig):
    global new_geohashes
    df_orig["geohash_short"] = df_orig["geohash"].apply(lambda x: x[:6])
    new_geohashes = new_geohashes.union([geohash for geohash in set(df_orig["geohash_short"]) if geohash not in geohash_df.index])
    print(f"Found {len(new_geohashes)} new geohashes")
    df_train = df_orig[df_orig['geohash_short'].isin(geohash_train_df.index)]
    df_test = df_orig[df_orig['geohash_short'].isin(geohash_test_df.index)]
    return df_train, df_test

In [55]:
dset = Dataset.retrieve(name='labelbox_import_halo_puddle')
df = dset.to_dataframe()

2024-03-20 07:31:13,425 - APIRequestor - ERROR - API error received | error_code : 403, error_message : {'extra': {'role_name': 'token_exchange'}, 'message': 'Role token_exchange is required to make this API call'}
ERROR:APIRequestor:API error received | error_code : 403, error_message : {'extra': {'role_name': 'token_exchange'}, 'message': 'Role token_exchange is required to make this API call'}


In [78]:
print(len(df))
df['speed'] = df['gps_can_data'].apply(lambda x: x.get('speed', np.nan))
df = df[df["speed"] > 0.2]
df = df[df["speed"] < 25]
df['day'] = pd.to_datetime(df['collected_on']).dt.strftime('%Y-%m-%d')
df["geohash_8"] = df["geohash"].apply(lambda x: x[:8])
df["geohash_7"] = df["geohash"].apply(lambda x: x[:7])
df["geohash_6"] = df["geohash"].apply(lambda x: x[:6])
train_df, test_df = filter_df(df)
print(len(df), len(train_df), len(test_df))

33892


KeyError: 'gps_can_data'

In [ ]:
df_diverse = train_df.groupby(['geohash_6', 'day', 'camera_location', 'robot_name']).first()

In [82]:
ids = list(set(df_diverse['id']) - set(tire_ids))

In [83]:
Dataset.create(
    name=f'halo_water_images_{len(ids)}',
    description='Selected to include mostly images that have puddles in them.',
    kind=Dataset.KIND_IMAGE,
    image_ids=ids,
)

{}

In [43]:
# May need to unactivate jupyter notebook, switch envs, run jupyter notebook again
# !conda activate jupiterdata39

In [16]:
# import sys
# sys.path.append('/home/alex.li/git/JupiterData')
from jupiterdata.config import dataset_config
from jupiterdata.utils.dataset import imageids_to_dataset
imageids_to_dataset(
    ids,
    dataset_name=f"{dset.name}_diverse",
    dataset_description=f"{dset.description}, choosing one from each day/camera location/robot/geohash to get {len(df_diverse)} left ids",
    dataset_kind = "image",
    mode = "stereo",
    annotation_state = dataset_config.VALID_ANNOTATION_STATES,
    camera_location = dataset_config.LEFT_CAMERAS,
    production_dataset = False,
    camera_pairs_list = dataset_config.ALL_CAMERA_PAIRS_LIST
)

ModuleNotFoundError: No module named 'pyspark'

In [ ]:
# from scripts.notebooks.manny.diversify import diversify_dataset
# diversify_dataset('labelbox_import_tire_tracks_100k', 30000)

# Create final Dataset

In [67]:
LEFT_CAMERAS = ['T01','T02','T05','T06','T09','T10', 'T13', 'T14']

In [68]:
df = pd.read_csv('/mnt/sandbox1/alex.li/introspection/25080_alleyson/labelbox_import_tire_tracks_100k_diverse/output.csv')
df = df[df['camera_location'].isin(LEFT_CAMERAS)]
df = df.drop_duplicates(['id'])

In [81]:
fp_df = df[df['state'] == 'false_positive']
ids = list(fp_df['id'])
tire_ids = ids

In [72]:
Dataset.create(
    name=f'tire_track_model_fps_{len(ids)}',
    description='Selected to include mostly images that have tires in them. Only images where some object was detected by the model',
    kind=Dataset.KIND_IMAGE,
    image_ids=ids,
)

{}